# Определение уровня английского языка в фильмах по субтитрам

## Описание работы

### Постановка задачи

**Заказчик:** Языковой центр.  
Необходимо разработать приложение которое позволит определить какой уровень знания языка необходим студенту, чтобы посмотреть выбранный фильм.

**Дано:**  
movies_labels.xlsx - Фильм и его языковой уровень.  
levels.xlsx - перечень слов для определения уровня знания английского языка

Папка Subtitles_all содержащая набор субтитров для размеченных фильмов

### Цели и этапы решения задачи

**Цель:** реализовать модель позволяющую оценить уровень аглийского языка в фильме.

**Этапы решения задачи:**
1.	Загрузить данные
2.	Обработать текстовые данные: убрать заглавные буквы, отчистить данные от стоп слов, удалить символы, пунктуацию, сделать стемминг и лемматизацию.
3.	Проверить данные на наличие дубликатов
4.	Обогатить данные посчитав количество слов каждой категории в субтитре.
5.	Разделить данные на обучающую и тестовую
6.  Построить модель и получить результат. Определяющей метрикой является accuracy.
7.  Написать вывод

## Подготовка данных

### Импорт библиотек¶

In [12]:
!pip install pysrt

In [13]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pysrt
import os
import re
import string
import nltk

from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


random_state = 123456

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Fenya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Fenya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Fenya\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Загрузка обработка данных с разметкой фильмов

#### Загрузка данных

In [14]:
df = pd.read_excel(r'movies_labels.xlsx', index_col=0)  
df['Movie'] = df['Movie'].str.replace(':', '')
df.head(5)

,Movie,Level
id,,
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2/A2+


#### Удаление дубликатов

In [15]:
df[df['Movie'].duplicated()]['Movie'].values
for movie in df[df['Movie'].duplicated()]['Movie'].values:
    display(df[df['Movie'] == movie])

,Movie,Level
id,,
43,Inside_out(2015),B1
44,Inside_out(2015),B1


,Movie,Level
id,,
38,Powder(1995),B1
68,Powder(1995),B1


,Movie,Level
id,,
75,The_blind_side(2009),B2
84,The_blind_side(2009),B1


,Movie,Level
id,,
83,The_terminal(2004),B1
99,The_terminal(2004),"A2/A2+, B1"


In [16]:
df.loc[df['Movie']=='The_blind_side(2009)', 'Level'] = 'B2'
df.loc[df['Movie']=='The_terminal(2004)', 'Level'] = 'A2/A2+, B1'
df.loc[df['Movie']=='The_terminal(2004)']

df = df.drop_duplicates().reset_index(drop=True)

df.duplicated().sum()

0

### Загрузка набора слов для уровней

In [17]:
file_path = 'levels.xlsx'
level_words = pd.read_excel(file_path, sheet_name='All', usecols=[0, 1, 2, 3, 4])
A1 = level_words['A1'].unique()[:-1]
A2 = level_words['A2'].unique()[:-1]
B1 = level_words['B1'].unique()[:-1]
B2 = level_words['B2'].unique()
C1 = level_words['C1'].unique()[:-1]

### Загрузка и обработка субтитров

In [18]:
def clean_string(text):

    final_string = ""

    # Make lower
    text = text.lower()

    # Remove line breaks
    text = re.sub(r'\n', ' ', text)
    
    # Remove simbols
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    
    # Remove numbers
    nums = ['0','1','2','3','4','5','6','7','8','9']
    for num in nums: text = re.sub(num, ' ', text) 

    # Remove puncuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)  
         
    # Remove stop words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")
    useless_words = useless_words + ['hi', 'im']

    text_filtered = [word for word in text if not word in useless_words]
    
    # Stemming
    stemmer = PorterStemmer() 
    text_stemmed = [stemmer.stem(y) for y in text_filtered]
    
    # Lemmatization
    lem = WordNetLemmatizer()
    text_stemmed = [lem.lemmatize(y) for y in text_stemmed]
    
    final_string = ' '.join(text_stemmed)
    
    return final_string

In [19]:
def get_sub_text(path):
    input_files_path = path+"\\"
    # Collection of files in a folder
    files: list = os.listdir(input_files_path)
    files = sorted(files)
    name = []
    subs = []
    
    for file in files:
        #open and encoding file
        sub = pysrt.open(input_files_path + file, encoding='iso-8859-1')
        #get text
        subtitles_text = sub.text
        #cleaning text
        subtitles_text = clean_string(subtitles_text)
        name.append(file[:-4])
        subs.append(subtitles_text)
    return name,subs

movie = []
subtitles = []

links = ['Subtitles_all\A2',
         'Subtitles_all\B1',
         'Subtitles_all\B2',
         'Subtitles_all\C1',
         'Subtitles_all\Subtitles']

for link in links:
    names, subs = get_sub_text(link)
    for name in names: movie.append(name)
    for sub in subs: subtitles.append(sub)

#create DataFrame with all subtitles
all_subtitles = pd.DataFrame()
all_subtitles['Movie'] = movie
all_subtitles['Subtitles'] = subtitles

### Создание общего DataFrame

In [20]:
level_df = df.merge(all_subtitles, on='Movie', how='left')
level_df.head(5)

,Movie,Level,Subtitles
0,10_Cloverfield_lane(2016),B1,font color ffff b fix sync bozxphd enjoy flick...
1,10_things_I_hate_about_you(1999),B1,hey right cameron go nine school year armi bra...
2,A_knights_tale(2001),B2,resync xenzai nef retail help due list two min...
3,A_star_is_born(2018),B2,font color ffffff sync correct font b font col...
4,Aladdin(1992),A2/A2+,oh come land faraway place caravan camel roam ...


### Обогощение данных

Данные можно обогатить с использованием перечней слов из файла levels.xlsx. Для каждого фильма рассчитывается сколько слова каждой категории содержит субтитр фильма

In [21]:
def counter_level(sub_wors):
    #unique sets for subtitles 
    words = list(set(sub_wors))
    #intersection of word sets 
    A1_counter=len(set(words).intersection(set(A1)))
    A2_counter=len(set(words).intersection(set(A2)))
    B1_counter=len(set(words).intersection(set(B1)))
    B2_counter=len(set(words).intersection(set(B2)))
    C1_counter=len(set(words).intersection(set(C1)))

    return A1_counter, A2_counter, B1_counter, B2_counter, C1_counter

In [22]:
A1_counter_list = []
A2_counter_list = []
B1_counter_list = []
B2_counter_list = []
C1_counter_list = []
for i in range(level_df.shape[0]):
    A1_counter, A2_counter, B1_counter, B2_counter, C1_counter = counter_level(level_df['Subtitles'][i].split())
    A1_counter_list.append(A1_counter)
    A2_counter_list.append(A2_counter)
    B1_counter_list.append(B1_counter)
    B2_counter_list.append(B2_counter)
    C1_counter_list.append(C1_counter)

In [23]:
level_df['A1_counter'] = A1_counter_list
level_df['A2_counter'] = A2_counter_list
level_df['B1_counter'] = B1_counter_list
level_df['B2_counter'] = B2_counter_list
level_df['C1_counter'] = C1_counter_list

### Итоговый DataFrame

In [24]:
level_df['A2'] = 0 
level_df.loc[level_df['Level'].isin(['A2','A2/A2+','A2/A2+, B1']), 'A2'] = 1

level_df['B1'] = 0 
level_df.loc[level_df['Level'].isin(['B1','A2/A2+, B1','B1, B2']), 'B1'] = 1

level_df['B2'] = 0 
level_df.loc[level_df['Level'].isin(['B2', 'B1, B2']), 'B2'] = 1

level_df['C1'] = 0 
level_df.loc[level_df['Level'].isin(['C1']), 'C1'] = 1

level_df.head(5)

,Movie,Level,Subtitles,A1_counter,A2_counter,B1_counter,B2_counter,C1_counter,A2,B1,B2,C1
0,10_Cloverfield_lane(2016),B1,font color ffff b fix sync bozxphd enjoy flick...,254,131,95,84,38,0,1,0,0
1,10_things_I_hate_about_you(1999),B1,hey right cameron go nine school year armi bra...,295,165,125,108,54,0,1,0,0
2,A_knights_tale(2001),B2,resync xenzai nef retail help due list two min...,276,176,116,110,63,0,0,1,0
3,A_star_is_born(2018),B2,font color ffffff sync correct font b font col...,300,170,105,105,37,0,0,1,0
4,Aladdin(1992),A2/A2+,oh come land faraway place caravan camel roam ...,273,168,117,114,57,1,0,0,0


Итоговый датафрейм содержит данные о разметке фильмов, отчищенные субтитры, количество слова каждой категории в субтитре и бинарный переменные, которые будут при моделировании выступать как зависимая переменная.


## Моделирование

In [25]:
def res(model,X_train, y_train, X_test, y_test):
    dt = datetime.now()
    model = model.fit(X_train, y_train)
    dt_train = datetime.now() - dt

    pred_train = model.predict(X_train)
    pred_valid = model.predict(X_test)

    a_train = accuracy_score(y_train, pred_train)
    a_valid = accuracy_score(y_test, pred_valid)

    print('model name:', type(model).__name__)
    print('time:', dt_train)
    print('accuracy train: {}%'.format(round(a_train*100,2)))
    print('accuracy test: {}%'.format(round(a_valid*100,2))) 

Для работы с текстовыми данными воспользуемся генератором фичей TfidfVectorizer. А в качество модели возьмём MultinomialNB. 

Наивный байесовский классификатор для полиномиальных моделей подходит для классификации с дискретными признаками (например, количество слов для классификации текста), также в документации сказано, что метод работает с tf-idf векторизацией.

Для каждого уровня будет построена своя бинарная модель в двух вариантах:
1. Модель получает только матрицу полученную при векторизации субтитров
2. Модель обучается на данных о количестве слов в каждой категории и матрице векторизации субтитров 

In [26]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(level_df['Subtitles'])

tfidf_df = pd.DataFrame(X.toarray(), index=level_df['Movie'], columns = vectorizer.get_feature_names_out())
tfidf_level_df = level_df.merge(tfidf_df, on='Movie', how='left')
X_data = tfidf_level_df.drop(['Movie', 'Level','Subtitles', 'A2', 'B1', 'B2', 'C1'], axis=1)

### Модель для группы A2

In [27]:
y = level_df['A2']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = random_state,stratify=y)

#### Модель на текстовых данных

In [28]:
model = MultinomialNB()

parameters = {'alpha':[0.05, 0.3, 0.5, 0.7,0.9]}

RCmodel = RandomizedSearchCV(model, parameters, cv= 5, scoring='accuracy',n_iter=5)
RCmodel = RCmodel.fit(X_train, y_train)
print(RCmodel.best_params_)

model = MultinomialNB(alpha = RCmodel.best_params_['alpha'])
res(model,X_train, y_train, X_test, y_test)

{'alpha': 0.05}
model name: MultinomialNB
time: 0:00:00.001995
accuracy train: 89.42%
accuracy test: 85.42%


#### Модель на текстовых и количественных данных

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X_data,y, test_size = 0.2, random_state = random_state,stratify=y)

In [30]:
model = MultinomialNB()

parameters = {'alpha':[0.05, 0.3, 0.5, 0.7,0.9]}

RCmodel = RandomizedSearchCV(model, parameters, cv= 5, scoring='accuracy',n_iter=5)
RCmodel = RCmodel.fit(X_train, y_train)
print(RCmodel.best_params_)

model = MultinomialNB(alpha = RCmodel.best_params_['alpha'])
res(model,X_train, y_train, X_test, y_test)

{'alpha': 0.05}
model name: MultinomialNB
time: 0:00:00.104999
accuracy train: 84.13%
accuracy test: 83.33%


### Модель для группы B1

#### Модель на текстовых данных

In [31]:
y = level_df['B1']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = random_state,stratify=y)

In [32]:
model = MultinomialNB()

parameters = {'alpha':[0.05, 0.3, 0.5, 0.7,0.9]}

RCmodel = RandomizedSearchCV(model, parameters, cv= 5, scoring='accuracy',n_iter=5)
RCmodel = RCmodel.fit(X_train, y_train)
print(RCmodel.best_params_)

model = MultinomialNB(alpha = RCmodel.best_params_['alpha'])
res(model,X_train, y_train, X_test, y_test)

{'alpha': 0.05}
model name: MultinomialNB
time: 0:00:00.002002
accuracy train: 94.18%
accuracy test: 72.92%


#### Модель на текстовых и количественных данных

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X_data,y, test_size = 0.2, random_state = random_state,stratify=y)

In [34]:
model = MultinomialNB()

parameters = {'alpha':[0.05, 0.3, 0.5, 0.7,0.9]}

RCmodel = RandomizedSearchCV(model, parameters, cv= 5, scoring='accuracy',n_iter=5)
RCmodel = RCmodel.fit(X_train, y_train)
print(RCmodel.best_params_)

model = MultinomialNB(alpha = RCmodel.best_params_['alpha'])
res(model,X_train, y_train, X_test, y_test)

{'alpha': 0.05}
model name: MultinomialNB
time: 0:00:00.105000
accuracy train: 77.25%
accuracy test: 75.0%


### Модель для группы B2

#### Модель на текстовых данных

In [35]:
y = level_df['B2']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = random_state,stratify=y)

In [36]:
model = MultinomialNB()

parameters = {'alpha':[0.05, 0.3, 0.5, 0.7,0.9]}

RCmodel = RandomizedSearchCV(model, parameters, cv= 5, scoring='accuracy',n_iter=5)
RCmodel = RCmodel.fit(X_train, y_train)
print(RCmodel.best_params_)

model = MultinomialNB(alpha = RCmodel.best_params_['alpha'])
res(model,X_train, y_train, X_test, y_test)

{'alpha': 0.05}
model name: MultinomialNB
time: 0:00:00.002000
accuracy train: 94.71%
accuracy test: 77.08%


#### Модель на текстовых и количественных данных

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_data,y, test_size = 0.2, random_state = random_state,stratify=y)

In [38]:
model = MultinomialNB()

parameters = {'alpha':[0.05, 0.3, 0.5, 0.7,0.9]}

RCmodel = RandomizedSearchCV(model, parameters, cv= 5, scoring='accuracy',n_iter=5)
RCmodel = RCmodel.fit(X_train, y_train)
print(RCmodel.best_params_)

model = MultinomialNB(alpha = RCmodel.best_params_['alpha'])
res(model,X_train, y_train, X_test, y_test)

{'alpha': 0.05}
model name: MultinomialNB
time: 0:00:00.104000
accuracy train: 96.3%
accuracy test: 81.25%


### Модель для группы С1

#### Модель на текстовых данных

In [39]:
y = level_df['C1']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = random_state,stratify=y)

In [40]:
model = MultinomialNB()

parameters = {'alpha':[0.05, 0.3, 0.5, 0.7,0.9]}

RCmodel = RandomizedSearchCV(model, parameters, cv= 5, scoring='accuracy',n_iter=5)
RCmodel = RCmodel.fit(X_train, y_train)
print(RCmodel.best_params_)

model = MultinomialNB(alpha = RCmodel.best_params_['alpha'])
res(model,X_train, y_train, X_test, y_test)

{'alpha': 0.05}
model name: MultinomialNB
time: 0:00:00.001999
accuracy train: 88.36%
accuracy test: 83.33%


#### Модель на текстовых и количественных данных

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X_data,y, test_size = 0.2, random_state = random_state,stratify=y)

In [42]:
model = MultinomialNB()

parameters = {'alpha':[0.05, 0.3, 0.5, 0.7,0.9]}

RCmodel = RandomizedSearchCV(model, parameters, cv= 5, scoring='accuracy',n_iter=5)
RCmodel = RCmodel.fit(X_train, y_train)
print(RCmodel.best_params_)

model = MultinomialNB(alpha = RCmodel.best_params_['alpha'])
res(model,X_train, y_train, X_test, y_test)

{'alpha': 0.05}
model name: MultinomialNB
time: 0:00:00.107000
accuracy train: 83.07%
accuracy test: 83.33%


## Итоги

В результате проделанной работы были сделаны следующие шаги:

1. Загружены входные данные
2. Данные обработаны следующими методами:
- Удалены дубликаты
- Проведена чистка от мусорных слов и символов
- Проведена лемматизация
- Проведен стэминг
3. Данные были обогащены
4. Была расчитана модель MultinomialNB	
5. При помощи метода RandomizedSearchCV с помощью метрики accuracy были выбраны лучшие параметры для моделей.
 
В результате проделанной работы была были получены следующие значения метрик:

| name | train стандарт | train обогащ | test стандарт | test обогащ |
|----------|----------|----------|----------|----------|
| A2     | 89.42%   | 96.83%   | 85.42%   | 85.42%   |
| B1     | 77.25%   |94.71%   | 75.0%   | 77.08%   |
| B2     | 94.71%   | 96.3%   | 77.08%   | 81.25%   |
| C1     | 88.36%   | 83.07%   | 83.33%   | 83.33%   |

Не смотря на то, что результаты получены очень близкие видно, что на тестовой выборке модель на обогащенных данных ведет себя лучше или не уступает для всех уровней.

Модель на обогащенных данных показывает хороший стабильный результат для всех категорий фильмов и может быть использована заказчиком.